In [2]:
import qiskit
from qiskit import IBMQ
from qiskit import Aer
from qiskit import QuantumCircuit, ClassicalRegister,QuantumRegister, QiskitError
from qiskit.tools.visualization import circuit_drawer
from qiskit.tools.visualization import plot_histogram
from qiskit.providers.aer import noise
import random
import matplotlib.pyplot as plt
import numpy as np
import collections as c
import math
from math import *


In [3]:
## Defining some auxiliary functions
def convertToNumEncoding(genome_str):
    bin_str = ""
    for i in range(0, len(genome_str)):
        if genome_str[i] == 'A':
            bin_str = bin_str + "0"
        elif genome_str[i] == 'C':
            bin_str = bin_str + "1"
        elif genome_str[i] == 'G':
            bin_str = bin_str + "2"
        elif genome_str[i] == 'T':
            bin_str = bin_str + "3"
    return bin_str

In [6]:
## Initializing global variables
# Alphabet set (0, 1, 2, 3) := {A, C, G, T} for DNA Nucleotide bases
AS = {'00', '01', '10', '11'}
A = len(AS)
genome_file = open("Sars_cov_2.txt", "r")
#genome_file = open("Carsonella_rudii.txt", "r")
#genome_file = open("nsp14.txt", "r")
RG = genome_file.read()
LenRG=len(RG)
print('Length:', LenRG)
genome_file.close()
RG = convertToNumEncoding(RG)
RG = RG[0:LenRG]
N = len(RG)

Length: 1709


In [7]:
# Short Read search string
SR = '23?'
M = len(SR)
Q_A = ceil(log2(A))
# Data Qubits
Q_D = Q_A * M
# Tag Qubits
Q_T = ceil(log2(N - M + 1))
# Ancilla Qubits
Q_anc = 8
# Ancilla qubits ids
anc = []
for qi in range(0, Q_anc):
    anc.append(Q_D + Q_T + qi)
# Total number of qubits
Q = Q_D + Q_T + Q_anc

In [8]:
#Start codon
Start = '032'
Xl = len(Start)

#Stop codon
Stop = ['302','320','300']

In [9]:
IBMQ.save_account('APIKEY', overwrite=True)
## Initialization of IBM QX
IBMQ.enable_account('5b3b5238d1fec1257d958a113fb4936b7cb56a3bd4c1522b5b8c13366faca29feb421a44c3fb07bcf747db804e0e31a5f64694bcf54ba2c1e25310615110bc54')
#IBMQ.enable_account('bf333c31a0a3b9e5829fa0a0e0097216062f044a366ff7cce49dc0f004ad35da0d8067cb98c1f59e8d79704b055dac79de9e88c4e10306dc2cb14e20183f4e68')
#provider = IBMQ.get_provider()
#backend = provider.get_backend('ibmq_qasm_simulator')
backend = Aer.get_backend('qasm_simulator')

In [12]:
## Defining functions
def QAM():
    print('Reference genome:', RG, 'Sars_cov_2', 'Length of the genome:', LenRG)
    #print('Carsonella rudii:', RG1)
    #print('Chosen pattern for testing:', SR)
    print('Total number of qubits:', Q)
    print('Number of ancilla qubits:', Q_anc)
    qr = qiskit.QuantumRegister(Q)
    cr = qiskit.ClassicalRegister(Q_T)
    qc = qiskit.QuantumCircuit(qr, cr)
    # Patterns are stored
    generateInitialState(qc, qr)
# Patterns are turned into Hamming Distances
    evolveToHammingDistances(qc, qr)
# Marking the zero Hamming Distance states
    markZeroHammingDistance(qc, qr)
    inversionAboutMean(qc, qr)
# Applying again this funcions turns back the Hamming Distances into the original patterns
    evolveToHammingDistances(qc, qr)
# Marking the stored patterns
    markStoredPatterns(qc, qr)
    # From patterns to Hamming Distances again
    evolveToHammingDistances(qc, qr)
    inversionAboutMean(qc, qr)
    # Grover’s iterations
    it = 0
    for i in range(0, 1):
        markZeroHammingDistance(qc, qr)
        inversionAboutMean(qc, qr)
        it = it + 1
    print("Grover’s algorithm had {} iterations.".format(int(it)))
    finalGroverMeasurement(qc, qr, cr)
    QAM2(qc, qr)
    return qc

In [13]:
# Initialization as suggested by L. C. L. Hollenberg
def generateInitialState(qc, qr):
    for qi in range(0, Q_T):
        qc.h(qr[qi])
    
    control_qubits = []
    for ci in range(0, Q_T):
        control_qubits.append(qr[ci])
    
    ancilla_qubits = []
    for qi in anc:
        ancilla_qubits.append(qr[qi])
    
    for qi in range(0, N - M + 1):
        qis = format(qi, '0' + str(Q_T) + 'b')
        for qisi in range(0, Q_T):
            if qis[qisi] == '0':
                qc.x(qr[qisi])
        wMi = RG[qi:qi + M]
        #print("Tag: {} - Data: {}".format(qis, wMi))

        for wisi in range(0, M):
            wisia = format(int(wMi[wisi]), '0' + str(Q_A) + 'b')
            for wisiai in range(0, Q_A):
                if wisia[wisiai] == '1':
                    qc.mct(control_qubits, qr[Q_T + wisi * Q_A + wisiai], ancilla_qubits)
        for qisi in range(0,Q_T):
            if qis[qisi] == '0':
                qc.x(qr[qisi])
                
    return

In [14]:
def QAM2(qc, qr):
    ## To compare given search string (GD)
    control_qubits = []
    for qi in range(1, Q_D + Q_T):
        control_qubits.append(qr[qi])
    
    ancilla_qubits = []
    for qi in anc:
        ancilla_qubits.append(qr[qi])
    Mark = []
    Mark1= []
    for qi in range(0, N - M + 1):
        #print("1", qi)
        Seq = []
        
        qis = format(qi, '0' + str(Q_T) + 'b')
        for qisi in range(0, Q_T):
            if qis[qisi] == '0':
                qc.x(qr[qisi])
        wMi = RG[qi:qi + M]
        #print("1",wMi)
        counter = compare(wMi, Start)
        #print(counter, "yes")
        
        if counter == 3 :
            #print(Start, "found at ", qis)
            y = int(qis, 2)
            
            #print("y",y)
            
            for qi in range(y, N - M + 1, 3):
                qis = format(qi, '0' + str(Q_T) + 'b')
                for qisi in range(0, Q_T):
                    if qis[qisi] == '0':
                        qc.x(qr[qisi])
                wmi = RG[qi:qi + M]
                
                #print("Tag: {} - Data: {}".format(qis, wmi))
                Mark.append(int(qis,2))
                for i in range(0,3):
                    StopC = compare(wmi, Stop[i])
                    if StopC == 3:
                        break
                Seq.append(wmi)
                if StopC == 3:
                    break
                
            z=int(qis,2)+3
            Length=len(Seq)
            if Length > 30:    
                print(Length, "\t codons long ORF found at \t", y, ":\t",y,"-",z)
##                
##    plt.plot(Mark[:],'ro')
##    
##    #plt.gca().invert_yaxis()
##    plt.xlabel('genome sequence')
##    plt.show()
    return

In [15]:
def compare(a,b):
    #print(a,b)
    c = 0
    for x, y in zip(a, b):
        if x == y:
            c+=1
            #print(x,y)
    return c

In [16]:
def evolveToHammingDistances(qc, qr):
    for pi in range(0, M):
        if SR[pi] == '?':
            continue
        ppi = format(int(SR[pi]), '0' + str(Q_A) + 'b')
        for ppii in range(0, Q_A):
            if ppi[ppii] == '1':
                qc.x(qr[Q_T + pi * Q_A + ppii])
    return

In [17]:

# Oracle to mark zero Hamming Distance
def markZeroHammingDistance(qc, qr):
    for qi in range(0, Q_D):
        qc.x(qr[Q_T + qi])
    control_qubits = []
    for mi in range(0, M):
        if SR[mi] == '?':
            for ai in range(0, Q_A):
                control_qubits.append(qr[Q_T + mi * Q_A + ai])
    
    ancilla_qubits = []
    for qi in anc:
        ancilla_qubits.append(qr[qi])
    target = control_qubits.pop()
    qc.h(target)
    qc.mct(control_qubits, target, ancilla_qubits)
    qc.h(target)
    for qi in range(0, Q_D):
        qc.x(qr[Q_T + qi])
    return

In [18]:

# Inversion about mean
def inversionAboutMean(qc, qr):
    for si in range(0, Q_D + Q_T):
        qc.h(qr[si])
        qc.x(qr[si])
    
    control_qubits = []
    for qi in range(1, Q_D + Q_T):
        control_qubits.append(qr[qi])
    
    ancilla_qubits = []
    for qi in anc:
        ancilla_qubits.append(qr[qi])
    qc.h(qr[0])
    qc.mct(control_qubits, qr[0], ancilla_qubits)
    qc.h(qr[0])
    for si in range(0, Q_D + Q_T):
        qc.x(qr[si])
        qc.h(qr[si])
    return

In [19]:
# Oracle to mark stored patterns
def markStoredPatterns(qc, qr):
    control_qubits = []
    for qi in range(1, Q_D + Q_T):
        control_qubits.append(qr[qi])
    
    ancilla_qubits = []
    for qi in anc:
        ancilla_qubits.append(qr[qi])
   
    for qi in range(0, N - M + 1):
        qis = format(qi, '0' + str(Q_T) + 'b')
        for qisi in range(0, Q_T):
            if qis[qisi] == '0':
                qc.x(qr[qisi])
        wMi = RG[qi:qi + M]
        for wisi in range(0, M):
            wisia = format(int(wMi[wisi]), '0' + str(Q_A) + 'b')
            for wisiai in range(0, Q_A):
                if wisia[wisiai] == '0':
                    qc.x(qr[Q_T + wisi * Q_A + wisiai])
        qc.h(qr[0])
        qc.mct(control_qubits, qr[0], ancilla_qubits)
        qc.h(qr[0])
        for wisi in range(0,M):
            wisia = format(int(wMi[wisi]), '0' + str(Q_A) + 'b')
            for wisiai in range(0, Q_A):
                if wisia[wisiai] == '0':
                    qc.x(qr[Q_T + wisi * Q_A + wisiai])
        for qisi in range(0, Q_T):
            if qis[qisi] == '0':
                qc.x(qr[qisi])
    return


In [20]:
# Final measurement
def finalGroverMeasurement(qc, qr, cr):
    for qi in range(0, Q_T):
        qc.measure(qr[qi], cr[qi])
    return

In [21]:
# Main function
if __name__ == '__main__':
# Printing some data for testing
    qc = QAM()
    print("Circuit depth: {}".format(qc.depth()))
# Total number of gates
    print("Number of gates: {}".format(len(qc.data)))
    gate_num = 1
    for item in qc.data:
        qb_list = ""
        for qb in item[1]:
            qb_list = qb_list + str(qb.index) + ", "
        qb_list = qb_list[:len(qb_list)-2]
        #print("#{}: {}, {}".format(gate_num, item[0].name, qb_list))
        gate_num = gate_num + 1

Reference genome: 0213200003230010220131333000203323023002230031013222330103113010102210113010101131023233201013000331000013200223330323233201030113221030113002201032011303020020131031313032032223333000032003303100233003223301113001032333031011121200200213030020103230123210322033221331203231202222323103213013020200213233223011003330113330102130223333313010223233001130233213230113010223303233203010113003003010203333311020233023213000110112113220203100333000101131030110133032301000220133113322003230232123030002033230100032330023201010133000003131313201020231230333231330322210103221333202332010313032002303333232000030220113202121011323323130323203020123211010321333311013213310201013303211323322103103313033220333203301231303003112333032033203233100100322223333010223001130100021001103203132303323100231103223003210103230213023323203210031032013022323130213231101202321333233002123233201322013033200303113030033223203200132002033003212213323020002233100101032233233000213210330330210201000331110